In [1]:
%load_ext autoreload
%autoreload 2

from primitives.primitives import *
import matplotlib.pyplot as plt
from numpy.linalg import eig

In [175]:

#\tilde\Gamma_i \tilde\Gamma_j (\Gamma^1_i\Gamma^2_j - \Gamma^2_i\Gamma^1_j)\Gamma^1_{i'}\Gamma^2_{j'}

a = TensorProduct(sy_gamma_5, sy_gamma_5) * (TensorProduct(sy_gamma_1, sy_gamma_2)-TensorProduct(sy_gamma_2, sy_gamma_1))
a

Matrix([
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0, -2.0*I,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0, 2.0*I, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0, 2.0*I,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0, -2.0*I, 0, 0],
[0, 0, 2.0*I,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0, -2.0*I, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,   

In [176]:
b = TensorProduct(sy_gamma_5, sy_id) + TensorProduct(sy_id, sy_gamma_5)
b

Matrix([
[2, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 2, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, -2,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, -2, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0, -2,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0, -2, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 2, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0,  0, 0, 0,  0,  0, 0, 0, 0, 0, 2]])

In [177]:
np.unique(np.round(np.linalg.eigvals(np.array(a, dtype=complex))), 1e-7)

(array([-2.+0.j,  0.+0.j,  2.+0.j]), array([1, 8, 0]))

In [178]:
np.unique(np.round(np.linalg.eigvals(np.array(b, dtype=complex))), 1e-7)

(array([-2.+0.j,  0.+0.j,  2.+0.j]), array([5, 1, 0]))

In [179]:
from scipy.linalg import eig
def find_unitary_transform(A, B):
    """
    Find the unitary matrix U such that U A U^dagger = B.

    Parameters:
    A (ndarray): Input matrix A.
    B (ndarray): Input matrix B.

    Returns:
    U (ndarray or None): Unitary matrix U if it exists, otherwise None.
    """
    A = np.array(A, complex)
    B = np.array(B, complex)
    
    # Get the eigenvalues and eigenvectors of A
    eigvals_A, eigvecs_A = eig(A)
    eigvals_B, eigvecs_B = eig(B)
    
    # Sort the eigenvalues and corresponding eigenvectors for comparison
    idx_A = np.argsort(eigvals_A)
    idx_B = np.argsort(eigvals_B)
    
    eigvals_A_sorted = eigvals_A[idx_A]
    eigvals_B_sorted = eigvals_B[idx_B]
    
    eigvecs_A_sorted = eigvecs_A[:, idx_A]
    eigvecs_B_sorted = eigvecs_B[:, idx_B]

    # Check if the sorted eigenvalues match
    if not np.allclose(eigvals_A_sorted, eigvals_B_sorted):
        print("not all eigenvalues match")
        return None

    eigvecs_A_sorted = np.linalg.qr(eigvecs_A_sorted)[0]
    eigvecs_B_sorted = np.linalg.qr(eigvecs_B_sorted)[0]

    # Construct the unitary matrix U
    U = eigvecs_B_sorted @ eigvecs_A_sorted.conj().T
    
    # Check if U is unitary
    if np.allclose(U @ U.conj().T, np.eye(U.shape[0])):
        return U
    else:
        print("not unitary")
        return U

In [180]:
U = find_unitary_transform(a, b)

In [181]:
np.allclose(U @ U.conj().T, np.eye(U.shape[0]), rtol=1e10)

True

In [182]:
np.allclose(np.array(b, dtype=complex), U @ np.array(a, dtype=complex) @ U.conj().T)

True

In [183]:
Matrix(U)

Matrix([
[                     0,                       0,   -3.70074341541719e-17 + 0.707106781186548*I,                                             0, 5.55111512312578e-17 + 1.00148357108136e-32*I,                                              0, -2.22044604925031e-16 + 1.30841038223648e-17*I,                                             0,    0.707106781186548 + 3.70074341541719e-17*I,                                             0,    0,   0, -1.30841038223648e-17 - 1.11022302462516e-16*I,                                             0,   0,   0],
[                   1.0,                       0,                                             0,                                             0,                                             0,                                              0,                                              0,                                             0,                                             0,                                             0,    0,   0,        